- 사회적 불안지수 만들어야함
- 이혼율, 자살률, 1인가구율 등 단위와 범위가 다른 변수들을 0~1 사이의 같은 스케일로 맞추기 위해 정규화를 진행함
- 연관성에 대해서는 상관관계를 분석함
- 자치구 기준으로 병합
- 봉사활동 지표는 역정규화 처리
- mean으로 정규화된 지표를 계산한다. 

In [264]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

# 필요한 패키지
import os

In [265]:
# 현재 경로 확인
os.getcwd()


'C:\\Users\\admin\\Desktop\\mid_project\\MidProject'

In [266]:

# 상위 폴더로 이동
os.chdir('../../')

In [267]:
os.chdir(r'C:/Users/admin/Desktop/mid_project/MidProject')

In [268]:
# 현재 경로 확인
os.getcwd()


'C:\\Users\\admin\\Desktop\\mid_project\\MidProject'

In [269]:
# 예시 파일 경로들
df_divorce = pd.read_csv('data/processed/혼인이혼_동별.csv')          # 이혼율
df_single = pd.read_csv('data/processed/1인가구합.csv')         # 1인가구
df_dropout = pd.read_csv('data/processed/2023 구별 학업 중단율.csv')       # 학업중단율
df_suicide = pd.read_csv('data/processed/자살율통계_서울시울시2023.csv')          # 자살률
df_volunteer = pd.read_csv('data/processed/2023_봉사활동_등록현황.csv')       # 봉사활동 수

# 데이터 조인 등

In [270]:
#데이터를 확인한다.
df_divorce.head()

,자치구별,2023,2023.1
0,자치구별,혼인,이혼
1,종로구,440,199
2,중구,595,197
3,용산구,1002,312
4,성동구,1280,340


In [271]:
# 컬럼 이름을 바꿔준다.
df_divorce = df_divorce.rename(columns={'2023': '혼인',
                                        '2023.1' : '이혼'})
# 그리고 0번 raw는 필요없기 때문에 빼고 만들들겠다. 
df_divorce = df_divorce[(df_divorce['자치구별'] != '자치구별')]

# 데이터프레임을 확인한다. 
df_divorce.head()

,자치구별,혼인,이혼
1,종로구,440,199
2,중구,595,197
3,용산구,1002,312
4,성동구,1280,340
5,광진구,1361,450


In [272]:
# 인덱스 초기화
df_divorce = df_divorce.reset_index(drop=True)

In [273]:
# 우선 혼인 컬럼의 경우 필요 없기 때문에 없애준다.
df_divorce.drop('혼인', axis=1, inplace=True) 
df_divorce.head()

,자치구별,이혼
0,종로구,199
1,중구,197
2,용산구,312
3,성동구,340
4,광진구,450


In [274]:
# 이후 인구수 데이터를 위해 봉사활동 데이터를 가져오겠다. 
# 인구소계, 자치구구 컬럼, 등록률만 가져온 데이터프레임을 만든다. 
volunteer_df = df_volunteer[['자치구', '인구소계', '등록률 (%)']]
# 가져와서 등록률을 알아보기 쉽게 하기 위해 이름을 봉사활동 등록률으로 변경하도록 하겠다. 
volunteer_df = volunteer_df.rename(columns={'등록률 (%)': '봉사활동 등록률'})
# 그리고 데이터를 확인하도록 하겠다. 
volunteer_df.head()

,자치구,인구소계,봉사활동 등록률
0,종로구,139417,45.09
1,중구,121312,33.53
2,용산구,213151,26.23
3,성동구,277361,27.77
4,광진구,335554,27.17


In [275]:
# 우선 inner join으로 자치구가 있는 raw만 묶어서 데이터프레임으로 만들겠다. 
# 봉사활동 데이터의 경우 컬럼명이 자치구이므로 자치구별로 바꿔준 뒤 진행하겠다. 
volunteer_df.rename(columns={'자치구': '자치구별'}, inplace=True) 
merged1_df = df_divorce.merge(volunteer_df, on='자치구별', how='inner')
merged1_df.head()

,자치구별,이혼,인구소계,봉사활동 등록률
0,종로구,199,139417,45.09
1,중구,197,121312,33.53
2,용산구,312,213151,26.23
3,성동구,340,277361,27.77
4,광진구,450,335554,27.17


In [276]:
# 이혼율을 계산하는 컬럼을 만들어주겠다. 
# 문자열로 이루어진 데이터라 계산이 되지 않아 정수값으로 변환한 뒤 진행한다. 
# 문자열을 숫자로 변환
merged1_df['이혼'] = merged1_df['이혼'].astype(int)
merged1_df['인구소계'] = merged1_df['인구소계'].astype(int)
# 이혼율을 계산한다. 
merged1_df['이혼율'] = (merged1_df['이혼'] / merged1_df['인구소계']) * 1000
# 그리고 둘째자리에서 반올림하겠다. 
merged1_df['이혼율'] = merged1_df['이혼율'].round(2)
# 데이터를 확인한다. 
merged1_df

,자치구별,이혼,인구소계,봉사활동 등록률,이혼율
0,종로구,199,139417,45.09,1.43
1,중구,197,121312,33.53,1.62
2,용산구,312,213151,26.23,1.46
3,성동구,340,277361,27.77,1.23
4,광진구,450,335554,27.17,1.34
5,동대문구,415,341149,26.51,1.22
6,중랑구,599,382155,28.52,1.57
7,성북구,522,425602,25.13,1.23
8,강북구,499,288113,21.90,1.73
9,도봉구,473,306948,26.68,1.54


In [277]:
# 이제 이혼건수는 필요없으니 없애주도록 하겠다.
merged1_df.drop('이혼', axis=1, inplace=True)
merged1_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율
0,종로구,139417,45.09,1.43
1,중구,121312,33.53,1.62
2,용산구,213151,26.23,1.46
3,성동구,277361,27.77,1.23
4,광진구,335554,27.17,1.34


In [278]:
# 첫번째 합쳐진 데이터를 csv 파일로 내보낸다. 
merged1_df.to_csv('members/조준혁/합친거1.csv', encoding='utf-8-sig', index=True)

In [279]:
# 다른 데이터프레임도 merge하도록 하겠다. 
# 우선 데이터를 확인한다
df_single.head()

,Unnamed: 0,자치구별,1인가구수(남자),1인가구수(여자)
0,0,강남구,33217,43592
1,1,강동구,29123,33820
2,2,강북구,24116,28543
3,3,강서구,47105,57404
4,4,관악구,79518,71227


In [280]:
df_single.drop('Unnamed: 0', axis=1, inplace=True) 
df_single.head()

,자치구별,1인가구수(남자),1인가구수(여자)
0,강남구,33217,43592
1,강동구,29123,33820
2,강북구,24116,28543
3,강서구,47105,57404
4,관악구,79518,71227


In [281]:
# 여자와 남자의 수는 쓸모 있을 것 같다. 그러므로 새로운 컬럼을 만들어서 합쳐주겠다. 
# '남자1인가구' + '여자1인가구' → '1인가구합' 생성
df_single['1인가구수(남녀총합)'] = df_single['1인가구수(남자)'] + df_single['1인가구수(여자)']

In [282]:
df_single.head()

,자치구별,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합)
0,강남구,33217,43592,76809
1,강동구,29123,33820,62943
2,강북구,24116,28543,52659
3,강서구,47105,57404,104509
4,관악구,79518,71227,150745


In [283]:
merged2_df = merged1_df.merge(df_single, on='자치구별', how='inner')
merged2_df

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합)
0,종로구,139417,45.09,1.43,14040,15294,29334
1,중구,121312,33.53,1.62,13574,14719,28293
2,용산구,213151,26.23,1.46,19436,23046,42482
3,성동구,277361,27.77,1.23,23464,24647,48111
4,광진구,335554,27.17,1.34,34355,37604,71959
5,동대문구,341149,26.51,1.22,34960,35999,70959
6,중랑구,382155,28.52,1.57,32454,34452,66906
7,성북구,425602,25.13,1.23,30867,38281,69148
8,강북구,288113,21.90,1.73,24116,28543,52659
9,도봉구,306948,26.68,1.54,18400,23125,41525


In [284]:
# csv로 내보낸다.
merged2_df.to_csv('members/조준혁/합친거2.csv', encoding='utf-8-sig', index=True)

### 학업중단률
- 교육 과정 중 학생이 학업을 지속하지 않고 중단하는 비율
- 학업중단율 = (학업중단자 수 / 학생 수) * 100

In [285]:
# 다음 데이터도 확인한다. 
# 학업중단율 데이터
df_dropout.head()

,Unnamed: 0,자치구별,초등학교,초등학교(학업중단자수 (명)),초등학교(학업중단율 (%)),중학교,중학교(학업중단자수 (명)),중학교(학업중단율 (%)),고등학교,고등학교(학업중단자수 (명)),고등학교(학업중단율 (%))
0,0,종로구,4645,97,2.09,2481,34,1.37,7365,161,2.19
1,1,중구,4728,125,2.64,1565,27,1.73,5260,142,2.70
2,2,용산구,6148,257,4.18,3105,61,1.96,4390,81,1.85
3,3,성동구,10101,193,1.91,4339,37,0.85,3648,61,1.67
4,4,광진구,12921,185,1.43,6674,72,1.08,7253,112,1.54


In [286]:
# csv 불러올때 부터 실수한 것 같다. 
# 인덱스 지워준다. 
df_dropout.drop('Unnamed: 0', axis=1, inplace=True) 
df_dropout.head()

,자치구별,초등학교,초등학교(학업중단자수 (명)),초등학교(학업중단율 (%)),중학교,중학교(학업중단자수 (명)),중학교(학업중단율 (%)),고등학교,고등학교(학업중단자수 (명)),고등학교(학업중단율 (%))
0,종로구,4645,97,2.09,2481,34,1.37,7365,161,2.19
1,중구,4728,125,2.64,1565,27,1.73,5260,142,2.70
2,용산구,6148,257,4.18,3105,61,1.96,4390,81,1.85
3,성동구,10101,193,1.91,4339,37,0.85,3648,61,1.67
4,광진구,12921,185,1.43,6674,72,1.08,7253,112,1.54


- 학업중단자 수는 학생수 대비 학업중단자 수이기 때문에 학교의 수는 중요하지 않으므로 뺀다. 
- 학업 중단자 수는 자치구별 학생 수 규모에 따라 왜곡될 수 있으므로, 학생 수 대비 중단자 수를 반영한 학업중단율을 분석 지표로 활용할 것이다.

In [287]:
# 학업 중단율만 가져온다. 
# 자치구별 컬럼은 merge에 필요하니 가져온다.
dropout_df = df_dropout[['자치구별','초등학교(학업중단율 (%))','중학교(학업중단율 (%))','고등학교(학업중단율 (%))']]
# 컬럼명이 너무 길어서 바꿔준다. 
dropout_df.columns = [col.replace('(학업중단율 (%))', ' 학업중단율') for col in dropout_df.columns]

In [288]:
dropout_df.head()

,자치구별,초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율
0,종로구,2.09,1.37,2.19
1,중구,2.64,1.73,2.70
2,용산구,4.18,1.96,1.85
3,성동구,1.91,0.85,1.67
4,광진구,1.43,1.08,1.54


In [289]:
# 또 조인한다. 
merged3_df = merged2_df.merge(dropout_df, on='자치구별', how='inner')
merged3_df

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54
5,동대문구,341149,26.51,1.22,34960,35999,70959,1.07,0.62,2.02
6,중랑구,382155,28.52,1.57,32454,34452,66906,0.59,0.26,1.57
7,성북구,425602,25.13,1.23,30867,38281,69148,1.12,0.94,1.77
8,강북구,288113,21.90,1.73,24116,28543,52659,0.58,0.67,1.84
9,도봉구,306948,26.68,1.54,18400,23125,41525,0.40,0.63,1.85


In [290]:
# csv로 내보낸다.
merged3_df.to_csv('members/조준혁/합친거3.csv', encoding='utf-8-sig', index=True)

In [291]:
df_suicide.head()

,Unnamed: 0,자치구별,사망자수 총합,남자,여자,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률)
0,0,종로구,34,20,14,24.5,29.9,19.5
1,1,중구,35,25,10,29.4,43.3,16.3
2,2,용산구,46,29,17,21.7,28.4,15.4
3,3,성동구,64,44,20,23.1,32.7,14.0
4,4,광진구,91,64,27,27.2,39.8,15.6


In [292]:
# 인덱스 지워준다. 
df_suicide.drop('Unnamed: 0', axis=1, inplace=True) 
df_suicide.head()

,자치구별,사망자수 총합,남자,여자,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률)
0,종로구,34,20,14,24.5,29.9,19.5
1,중구,35,25,10,29.4,43.3,16.3
2,용산구,46,29,17,21.7,28.4,15.4
3,성동구,64,44,20,23.1,32.7,14.0
4,광진구,91,64,27,27.2,39.8,15.6


- 자살률 데이터를 보면 대부분 **"인구 10만 명당 자살자 수"**로 표현되어 있는데,
- 그 이유는 국제 표준화된 비교 단위를 맞추기 위함.
- 자살 건수는 지역 인구 크기에 따라 좌우된된다.
- 따라서 서로 다른 규모의 지역 간 비교를 위해 인구 대비 비율화가 필요함.

소계 자살률= 
전체 인구수/
(남자 자살자 수+여자 자살자 수)
 ×100,000

In [293]:
# 사망자수 총합이랑 자살률 소계만 남겨두고... 남녀 데이터도 혹시 모르니까 남겨놓겠다.
suicide_df = df_suicide[['자치구별','소계(10만명당자살률)','남자(10만명당자살률)','여자(10만명당자살률)']]
suicide_df.head()

,자치구별,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률)
0,종로구,24.5,29.9,19.5
1,중구,29.4,43.3,16.3
2,용산구,21.7,28.4,15.4
3,성동구,23.1,32.7,14.0
4,광진구,27.2,39.8,15.6


In [294]:
# join 실시한다. 
merged4_df = merged3_df.merge(suicide_df, on='자치구별', how='inner')
merged4_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률)
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,24.5,29.9,19.5
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,29.4,43.3,16.3
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,21.7,28.4,15.4
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,23.1,32.7,14.0
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,27.2,39.8,15.6


In [295]:
# csv로 내보낸다.
merged4_df.to_csv('members/조준혁/합친거4.csv', encoding='utf-8-sig', index=True)

In [296]:
df_foreign = pd.read_csv('data/processed/등록외국인_현황_국적별_구별_2023_3분기.csv')
df_foreign.head()

,Unnamed: 0,소계(남자),소계(여자),한국계 중국인(남자),한국계 중국인(여자),중국(남자),중국(여자),미국(남자),미국(여자),타이완(남자),...,페 루(남자),페 루(여자),덴 마 크(남자),덴 마 크(여자),모 로 코(남자),모 로 코(여자),이 라 크(남자),이 라 크(여자),기 타(남자),기 타(여자)
0,강남구,2452,2882,156,263,467,466,467,446,63,...,1.0,2.0,5.0,3.0,3.0,11.0,0.0,0.0,75,97
1,강동구,1812,2256,688,652,188,422,79,130,26,...,2.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,17,22
2,강북구,1613,3036,360,398,264,738,51,46,51,...,0.0,1.0,0.0,0.0,1.0,5.0,0.0,0.0,24,54
3,강서구,2470,3169,860,859,329,683,89,112,119,...,0.0,1.0,1.0,2.0,4.0,10.0,0.0,1.0,48,41
4,관악구,6190,9220,2888,2679,914,1831,143,231,54,...,3.0,15.0,7.0,4.0,15.0,47.0,0.0,2.0,193,285


In [297]:
# 외국인 남녀합 컬럼을 만든다
df_foreign['등록외국인 소계'] = df_foreign['소계(남자)'] + df_foreign['소계(여자)']
df_foreign.head()

,Unnamed: 0,소계(남자),소계(여자),한국계 중국인(남자),한국계 중국인(여자),중국(남자),중국(여자),미국(남자),미국(여자),타이완(남자),...,페 루(여자),덴 마 크(남자),덴 마 크(여자),모 로 코(남자),모 로 코(여자),이 라 크(남자),이 라 크(여자),기 타(남자),기 타(여자),등록외국인 소계
0,강남구,2452,2882,156,263,467,466,467,446,63,...,2.0,5.0,3.0,3.0,11.0,0.0,0.0,75,97,5334
1,강동구,1812,2256,688,652,188,422,79,130,26,...,1.0,0.0,0.0,3.0,4.0,0.0,0.0,17,22,4068
2,강북구,1613,3036,360,398,264,738,51,46,51,...,1.0,0.0,0.0,1.0,5.0,0.0,0.0,24,54,4649
3,강서구,2470,3169,860,859,329,683,89,112,119,...,1.0,1.0,2.0,4.0,10.0,0.0,1.0,48,41,5639
4,관악구,6190,9220,2888,2679,914,1831,143,231,54,...,15.0,7.0,4.0,15.0,47.0,0.0,2.0,193,285,15410


- 이 병합된 데이터의 목적은 우선 큰 그림을 그리기 위한 상관관계 분석에 있으니 소계만을 가져와서 merge 시켜준다.  

In [298]:

df_foreign = df_foreign.rename(columns={'Unnamed: 0': '자치구별'})
foreign_df = df_foreign[['자치구별','등록외국인 소계','소계(남자)','소계(여자)']]
foreign_df.head()

,자치구별,등록외국인 소계,소계(남자),소계(여자)
0,강남구,5334,2452,2882
1,강동구,4068,1812,2256
2,강북구,4649,1613,3036
3,강서구,5639,2470,3169
4,관악구,15410,6190,9220


In [299]:
merge5_df = merged4_df.merge(foreign_df, on='자치구별', how='inner')
merge5_df

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률),등록외국인 소계,소계(남자),소계(여자)
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,24.5,29.9,19.5,11182,4597,6585
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,29.4,43.3,16.3,10420,4873,5547
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,21.7,28.4,15.4,14540,7842,6698
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,23.1,32.7,14.0,7177,2966,4211
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,27.2,39.8,15.6,15355,6167,9188
5,동대문구,341149,26.51,1.22,34960,35999,70959,1.07,0.62,2.02,25.0,37.5,12.8,17875,6468,11407
6,중랑구,382155,28.52,1.57,32454,34452,66906,0.59,0.26,1.57,27.3,36.9,18.0,5143,2030,3113
7,성북구,425602,25.13,1.23,30867,38281,69148,1.12,0.94,1.77,22.3,28.4,16.7,12418,4497,7921
8,강북구,288113,21.90,1.73,24116,28543,52659,0.58,0.67,1.84,24.9,32.9,17.5,4649,1613,3036
9,도봉구,306948,26.68,1.54,18400,23125,41525,0.40,0.63,1.85,27.7,38.9,17.1,2393,823,1570


In [300]:
merge5_df = merge5_df.rename(columns={'소계(남자)': '외국인 남자 총합',
                                                '소계(여자)': '외국인 여자 총합'
                                                })
merge5_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률),등록외국인 소계,외국인 남자 총합,외국인 여자 총합
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,24.5,29.9,19.5,11182,4597,6585
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,29.4,43.3,16.3,10420,4873,5547
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,21.7,28.4,15.4,14540,7842,6698
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,23.1,32.7,14.0,7177,2966,4211
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,27.2,39.8,15.6,15355,6167,9188


In [301]:
merge5_df.dtypes

자치구별             object
인구소계              int32
봉사활동 등록률        float64
이혼율             float64
1인가구수(남자)         int64
1인가구수(여자)         int64
1인가구수(남녀총합)       int64
초등학교 학업중단율      float64
중학교 학업중단율       float64
고등학교 학업중단율      float64
소계(10만명당자살률)    float64
남자(10만명당자살률)    float64
여자(10만명당자살률)    float64
등록외국인 소계          int64
외국인 남자 총합         int64
외국인 여자 총합         int64
dtype: object

In [302]:
merge5_df.columns

Index(['자치구별', '인구소계', '봉사활동 등록률', '이혼율', '1인가구수(남자)', '1인가구수(여자)',
       '1인가구수(남녀총합)', '초등학교 학업중단율', '중학교 학업중단율', '고등학교 학업중단율', '소계(10만명당자살률)',
       '남자(10만명당자살률)', '여자(10만명당자살률)', '등록외국인 소계', '외국인 남자 총합', '외국인 여자 총합'],
      dtype='object')

In [303]:
merge5_df.to_csv('members/조준혁/합친거5.csv', encoding='utf-8-sig', index=True)

### 청소년 비행 데이터
- 기타 지역의 신고 접수건에 대해서는 자치구별 비교라는 본 프로젝트의 주제와 맞지 않아 사용하지 않음

In [304]:
df_flying = pd.read_csv('data/processed/서울시_청소년비행(구별매칭완료).csv')
df_flying.head()

,자치구,신고건수전체,신고성별남성,신고성별여성,신고성별불상,신고성별기타,접수긴급유형긴급,접수긴급유형중요,접수긴급유형일반,접수긴급유형민원,...,발생시간대14,발생시간대15,발생시간대16,발생시간대17,발생시간대18,발생시간대19,발생시간대20,발생시간대21,발생시간대22,발생시간대23
0,강남구,1194,586,254,354,0,0,76,1085,27,...,11,32,103,70,58,89,84,139,229,149
1,강동구,1072,570,210,292,0,1,60,982,23,...,35,55,98,86,92,98,128,101,128,82
2,강북구,760,365,168,227,0,2,64,671,20,...,12,32,64,51,57,50,84,76,75,59
3,강서구,1225,612,273,340,0,0,81,1117,17,...,30,52,106,99,68,93,99,151,171,103
4,관악구,606,314,149,143,0,2,64,521,19,...,9,24,54,40,24,37,57,68,79,53


In [305]:
flying_df = df_flying[['자치구','신고건수전체']]
flying_df.head()

,자치구,신고건수전체
0,강남구,1194
1,강동구,1072
2,강북구,760
3,강서구,1225
4,관악구,606


In [306]:
# 컬럼 이름을 다른 데이터에 맞춰 바꿔준다. 
flying_df = flying_df.rename(columns={'자치구': '자치구별',
                                     '신고건수전체' : '청소년비행 신고'})
flying_df

,자치구별,청소년비행 신고
0,강남구,1194
1,강동구,1072
2,강북구,760
3,강서구,1225
4,관악구,606
5,광진구,837
6,구로구,729
7,금천구,459
8,기타,8357
9,노원구,1417


In [307]:
merged6_df = merge5_df.merge(flying_df, on='자치구별', how='inner')

In [308]:
merged6_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률),등록외국인 소계,외국인 남자 총합,외국인 여자 총합,청소년비행 신고
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,24.5,29.9,19.5,11182,4597,6585,135
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,29.4,43.3,16.3,10420,4873,5547,179
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,21.7,28.4,15.4,14540,7842,6698,99
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,23.1,32.7,14.0,7177,2966,4211,345
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,27.2,39.8,15.6,15355,6167,9188,837


In [309]:
# csv로 내보낸다.
merged6_df.to_csv('members/조준혁/합친거6.csv', encoding='utf-8-sig', index=True)


### 범죄율 코드도 가져온다. 

In [317]:
# 범죄율 코드도 가져오기
df_crime = pd.read_csv('data/processed/transposed_seoul_df.csv')
df_crime.head()

,Unnamed: 0,살인기수,살인미수등,강도,강간,유사강간,강제추행,기타 강간 강제추행등,방화,절도범죄,...,마약범죄,보건범죄,환경범죄,교통범죄,노동범죄,안보범죄,선거범죄,병역범죄,기타범죄,총 범죄 발생 건수
0,종로구,2,1,5,25,10,204,0,4,1173,...,100,119,1,657,4,11,2,22,1650,9223
1,중구,0,3,6,30,4,182,0,13,1523,...,73,108,0,652,3,4,3,18,1612,9793
2,용산구,1,2,3,45,4,195,0,16,971,...,200,102,1,1076,4,3,0,30,1817,9042
3,성동구,1,1,2,22,6,92,0,4,891,...,95,29,1,935,3,1,2,39,1283,7367
4,광진구,0,2,5,42,9,188,0,5,1619,...,100,53,0,850,1,0,5,65,1556,9688


In [318]:
# 컬럼 이름을 바꿔준다.
df_crime = df_crime.rename(columns={'Unnamed: 0': '자치구별'})

In [319]:
# 총범죄발생건수 컬럼만 가져오기
crime_df = df_crime[['자치구별','총 범죄 발생 건수']]


In [320]:
crime_df.head()

,자치구별,총 범죄 발생 건수
0,종로구,9223
1,중구,9793
2,용산구,9042
3,성동구,7367
4,광진구,9688


In [321]:
merged7_df = merged6_df.merge(crime_df, on='자치구별', how='inner')
merged7_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,소계(10만명당자살률),남자(10만명당자살률),여자(10만명당자살률),등록외국인 소계,외국인 남자 총합,외국인 여자 총합,청소년비행 신고,총 범죄 발생 건수
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,24.5,29.9,19.5,11182,4597,6585,135,9223
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,29.4,43.3,16.3,10420,4873,5547,179,9793
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,21.7,28.4,15.4,14540,7842,6698,99,9042
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,23.1,32.7,14.0,7177,2966,4211,345,7367
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,27.2,39.8,15.6,15355,6167,9188,837,9688


In [494]:
# 강력범죄 컬럼도 필요하니 추가해야함
df_crime.head()

,자치구별,살인기수,살인미수등,강도,강간,유사강간,강제추행,기타 강간 강제추행등,방화,절도범죄,...,보건범죄,환경범죄,교통범죄,노동범죄,안보범죄,선거범죄,병역범죄,기타범죄,총 범죄 발생 건수,강력범죄
0,종로구,2,1,5,25,10,204,0,4,1173,...,119,1,657,4,11,2,22,1650,9223,251
1,중구,0,3,6,30,4,182,0,13,1523,...,108,0,652,3,4,3,18,1612,9793,238
2,용산구,1,2,3,45,4,195,0,16,971,...,102,1,1076,4,3,0,30,1817,9042,266
3,성동구,1,1,2,22,6,92,0,4,891,...,29,1,935,3,1,2,39,1283,7367,128
4,광진구,0,2,5,42,9,188,0,5,1619,...,53,0,850,1,0,5,65,1556,9688,251


In [336]:
# 강력범죄에 포함될 컬럼들
crime_cols = ['살인기수', '살인미수등', '강도', '강간', '유사강간', '강제추행', '기타 강간 강제추행등', '방화']

# 컬럼 합산하여 '강력범죄'라는 새로운 컬럼 생성
df_crime['강력범죄'] = df_crime[crime_cols].sum(axis=1)

# 결과 확인
print(df_crime[['자치구별', '강력범죄']].head())

  자치구별  강력범죄
0  종로구   251
1   중구   238
2  용산구   266
3  성동구   128
4  광진구   251


In [337]:
# crime_df1을 만든다
crime_df1 = df_crime[['자치구별', '강력범죄']]
crime_df1.head()

,자치구별,강력범죄
0,종로구,251
1,중구,238
2,용산구,266
3,성동구,128
4,광진구,251


In [338]:
# 이후 merge
merged8_df = merged7_df.merge(crime_df1, on='자치구별', how='inner')
# merged8_df.head()

In [339]:
# merged8_df.head()

In [ ]:

# 가구수에 대한 전처리도 진행하도록 함
df_home = pd.read_csv('data/processed/인구__가구_및_주택_시군구.csv')
# df_home.head()

In [354]:
# 데이터 확인 결과 가구-계 (가구) 라는 컬럼에 자치구별 가구 수 통계가 있었음
home_df = df_home[['자치구별','가구-계 (가구)']]
# home_df.head()

In [355]:
merged9_df = merged8_df.merge(home_df, on='자치구별', how='inner')
# merged9_df.head()

In [356]:
# csv로 내보낸다.
merged9_df.to_csv('members/조준혁/합친거9.csv', encoding='utf-8-sig', index=True)

In [357]:
merged9_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,...,강력범죄,인구 비율,청소년비행_비율,1인가구_비율,1인가구_비율(여성),1인가구_비율(남성),외국인비율,총범죄율,강력범죄율,가구-계 (가구)
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,...,251,1.485366,96.831807,0.682437,0.355805,0.326632,8.020543,6615.405582,180.035433,70217
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,...,238,1.292473,147.553416,0.658219,0.342428,0.315790,8.589422,8072.573200,196.188341,64325
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,...,266,2.270938,46.445947,0.988317,0.536150,0.452166,6.821455,4242.063138,124.794160,103826
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,...,128,2.955039,124.386630,1.119272,0.573397,0.545875,2.587602,2656.105220,46.149242,127950
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,...,251,3.575035,249.438242,1.674080,0.874833,0.799247,4.576015,2887.165702,74.801671,168848


범죄율/강력범죄율은 실제 결과, 예측하고 싶은 대상이며 사회불안지수는 그 결과를 예측하거나 설명하기 위한 지표인데
포함하면 예측 대상이 원인 안에 섞이는 셈이니 포함하지 않는다. 

### 정신의료기관 및 병상 수, 우울감 경험률 데이터 추가

In [378]:
hospt_df = pd.read_csv('members/조준혁/정신의료기관 및 병상수 2023.csv', index_col = 0)

In [379]:
hospt_df

,자치구별,정신의료기관수,정신의료기관 병상 수
1,종로구,27,83
2,중구,21,20
3,용산구,9,21
4,성동구,17,33
5,광진구,25,342
6,동대문구,20,774
7,중랑구,18,341
8,성북구,14,34
9,강북구,10,0
10,도봉구,9,375


In [380]:
merged9_df = merged9_df.merge(hospt_df, on='자치구별', how='inner')

In [382]:
merged9_df.to_csv('members/조준혁/합친거9.csv', encoding='utf-8-sig', index=True)

In [383]:
merged9_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,...,1인가구_비율(여성),1인가구_비율(남성),외국인비율,총범죄율,강력범죄율,가구-계 (가구),정신의료기관수,정신의료기관 병상 수,정신의료기관비율,병상 수 비율
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,...,21.781050,19.995158,8.020543,6615.405582,180.035433,70217,27,83,19.366361,59.533629
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,...,22.882239,21.102215,8.589422,8072.573200,196.188341,64325,21,20,17.310736,16.486415
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,...,22.196752,18.719781,6.821455,4242.063138,124.794160,103826,9,21,4.222359,9.852171
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,...,19.262993,18.338413,2.587602,2656.105220,46.149242,127950,17,33,6.129196,11.897852
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,...,22.270918,20.346702,4.576015,2887.165702,74.801671,168848,25,342,7.450366,101.921002


In [384]:
df_blue = pd.read_excel('members/조준혁/SMHDB_통계DB_연령별 우울감 경험률(자치구).xlsx', skiprows=1)
df_blue.head()

,지역,전체,19세 이상 29세 이하,30세 이상 39세 이하,40세 이상 49세 이하,50세 이상 59세 이하,60세 이상 69세 이하,70세 이상
0,서울시,8.4,8.8,8.0,7.9,8.6,7.7,9.1
1,종로구,7.2,4.4,9.7,13.0,5.7,4.1,6.8
2,중구,6.9,6.1,6.4,10.9,4.1,7.6,6.7
3,용산구,7.0,12.0,5.6,6.7,4.9,5.0,7.8
4,성동구,7.0,10.9,4.5,9.7,6.0,5.2,5.4


#### 우울감 경험률: 최근 1년 동안 연속적으로 2주 이상 일상생활에 지장이 있을 정도의 우울감(슬픔이나 절망감 등)을 경험한 사람의 분율

In [385]:
# 컬럼 이름을 바꿔준다.
df_blue = df_blue.rename(columns={'지역': '자치구별',
                                  '전체' : '우울감경험율'})
blue_df = df_blue[['자치구별','우울감경험율']]
blue_df.head()

,자치구별,우울감경험율
0,서울시,8.4
1,종로구,7.2
2,중구,6.9
3,용산구,7.0
4,성동구,7.0


In [ ]:
#join
merged10_df = merged9_df.merge(blue_df, on='자치구별', how='inner')


,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,...,1인가구_비율(남성),외국인비율,총범죄율,강력범죄율,가구-계 (가구),정신의료기관수,정신의료기관 병상 수,정신의료기관비율,병상 수 비율,우울감경험율
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,...,19.995158,8.020543,6615.405582,180.035433,70217,27,83,19.366361,59.533629,7.2
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,...,21.102215,8.589422,8072.573200,196.188341,64325,21,20,17.310736,16.486415,6.9
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,...,18.719781,6.821455,4242.063138,124.794160,103826,9,21,4.222359,9.852171,7.0
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,...,18.338413,2.587602,2656.105220,46.149242,127950,17,33,6.129196,11.897852,7.0
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,...,20.346702,4.576015,2887.165702,74.801671,168848,25,342,7.450366,101.921002,10.2
5,동대문구,341149,26.51,1.22,34960,35999,70959,1.07,0.62,2.02,...,20.439783,5.239646,2897.560890,49.538471,171039,20,774,5.862541,226.880337,4.8
6,중랑구,382155,28.52,1.57,32454,34452,66906,0.59,0.26,1.57,...,18.559672,1.345789,2495.322579,46.839633,174863,18,341,4.710131,89.230809,7.7
7,성북구,425602,25.13,1.23,30867,38281,69148,1.12,0.94,1.77,...,16.146869,2.917749,1910.235384,34.774273,191164,14,34,3.289458,7.988684,8.6
8,강북구,288113,21.90,1.73,24116,28543,52659,0.58,0.67,1.84,...,17.938381,1.613603,2793.348443,52.409992,134438,10,0,3.470860,0.000000,8.4
9,도봉구,306948,26.68,1.54,18400,23125,41525,0.40,0.63,1.85,...,14.113676,0.779611,1802.585454,35.185113,130370,9,375,2.932093,122.170531,11.3


In [407]:
merged10_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,...,1인가구_비율(남성),외국인비율,총범죄율,강력범죄율,가구-계 (가구),정신의료기관수,정신의료기관 병상 수,정신의료기관비율,병상 수 비율,우울감경험율
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,...,19.995158,8.020543,6615.405582,180.035433,70217,27,83,19.366361,59.533629,7.2
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,...,21.102215,8.589422,8072.573200,196.188341,64325,21,20,17.310736,16.486415,6.9
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,...,18.719781,6.821455,4242.063138,124.794160,103826,9,21,4.222359,9.852171,7.0
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,...,18.338413,2.587602,2656.105220,46.149242,127950,17,33,6.129196,11.897852,7.0
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,...,20.346702,4.576015,2887.165702,74.801671,168848,25,342,7.450366,101.921002,10.2


### 시군구 실업률 데이터

In [ ]:
# 시군구 실업률 데이터
# CSV 불러와서 확인
df_silup = pd.read_csv("data/raw/시군구_경제활동인구_총괄_20250602112953.csv", skiprows=1)
df_silup.head()

,행정구역별,실업률 (％),실업률 (％).1
0,서울 종로구,2.8,4.0
1,서울 중구,3.8,2.4
2,서울 용산구,3.4,3.6
3,서울 성동구,3.0,3.2
4,서울 광진구,3.3,4.1


In [411]:
df_silup['행정구역별'] = df_silup['행정구역별'].str.replace('서울 ', '', regex=False)

In [ ]:
# 2023 전체 실업률 계산 (평균)
df_silup['2023 실업률'] = (df_silup['실업률 (％)'] + df_silup['실업률 (％).1']) / 2

In [414]:

#컬럼이름 바꾼다.
df_silup = df_silup.rename(columns={'행정구역별': '자치구별'})
# 필요한 컬럼만 빼서 새로 데이터프레임 만든다. 
silup_df = df_silup[['자치구별','2023 실업률']]
# csv 내보낸다.
silup_df.to_csv('members/조준혁/2023 자치구별 실업률.csv', encoding='utf-8-sig', index=True)

In [415]:
# 실업률도 merge10에 더한다.
merged10_df = merged10_df.merge(silup_df, on='자치구별', how='inner')
# 확인 
merged10_df.head()

,자치구별,인구소계,봉사활동 등록률,이혼율,1인가구수(남자),1인가구수(여자),1인가구수(남녀총합),초등학교 학업중단율,중학교 학업중단율,고등학교 학업중단율,...,외국인비율,총범죄율,강력범죄율,가구-계 (가구),정신의료기관수,정신의료기관 병상 수,정신의료기관비율,병상 수 비율,우울감경험율,2023 실업률
0,종로구,139417,45.09,1.43,14040,15294,29334,2.09,1.37,2.19,...,8.020543,6615.405582,180.035433,70217,27,83,19.366361,59.533629,7.2,3.4
1,중구,121312,33.53,1.62,13574,14719,28293,2.64,1.73,2.70,...,8.589422,8072.573200,196.188341,64325,21,20,17.310736,16.486415,6.9,3.1
2,용산구,213151,26.23,1.46,19436,23046,42482,4.18,1.96,1.85,...,6.821455,4242.063138,124.794160,103826,9,21,4.222359,9.852171,7.0,3.5
3,성동구,277361,27.77,1.23,23464,24647,48111,1.91,0.85,1.67,...,2.587602,2656.105220,46.149242,127950,17,33,6.129196,11.897852,7.0,3.1
4,광진구,335554,27.17,1.34,34355,37604,71959,1.43,1.08,1.54,...,4.576015,2887.165702,74.801671,168848,25,342,7.450366,101.921002,10.2,3.7


- 결국 인구 수의 차이가 큰 영향을 준다는 것
- 그러므로 자치구별 인구수로 각자 나눠서 인구 수의 영향력을 줄이는 방향으로 결정 

- 가구 수는 자치구별 가구로 나누었음

- 정신의료기관비율, 병상 수 비율, 청소년비행_비율, 총범죄율, 강력범죄율은 인구 10만명을 기준으로 해서 비율화 해주었음.
- 가구비율과 외국인 비율은 그냥 백분율로. 

In [ ]:
# 비율화를 진행한다.
merged10_df['정신의료기관비율'] = (merged10_df['정신의료기관수'] / merged10_df['인구소계']) * 100000  # 10만명당
merged10_df['병상 수 비율'] = (merged10_df['정신의료기관 병상 수'] / merged10_df['인구소계']) * 100000  # 10만명당

merged10_df['인구 비율'] = (merged10_df['인구소계'] / 9386034)* 100

merged10_df['청소년비행_비율'] = (merged10_df['청소년비행 신고'] / merged10_df['인구소계']) * 100000  # 10만명당
merged10_df['1인가구_비율'] = (merged10_df['1인가구수(남녀총합)'] / merged10_df['가구-계 (가구)']) * 100
merged10_df['1인가구_비율(여성)'] = (merged10_df['1인가구수(여자)'] / merged10_df['가구-계 (가구)']) * 100
merged10_df['1인가구_비율(남성)'] = (merged10_df['1인가구수(남자)'] / merged10_df['가구-계 (가구)']) * 100
merged10_df['외국인비율'] = (merged10_df['등록외국인 소계'] / merged10_df['인구소계']) * 100


merged10_df['총범죄율'] = (merged10_df['총 범죄 발생 건수'] / merged10_df['인구소계']) * 100000 # 10만명당
merged10_df['강력범죄율'] = (merged10_df['강력범죄'] / merged10_df['인구소계']) * 100000 # 10만명당

In [416]:
# csv로 내보내기 
merged10_df.to_csv('members/조준혁/궁극머지데이터.csv', encoding='utf-8-sig', index=True)

In [417]:
merged10_df.columns

Index(['자치구별', '인구소계', '봉사활동 등록률', '이혼율', '1인가구수(남자)', '1인가구수(여자)',
       '1인가구수(남녀총합)', '초등학교 학업중단율', '중학교 학업중단율', '고등학교 학업중단율', '소계(10만명당자살률)',
       '남자(10만명당자살률)', '여자(10만명당자살률)', '등록외국인 소계', '외국인 남자 총합', '외국인 여자 총합',
       '청소년비행 신고', '총 범죄 발생 건수', '강력범죄', '인구 비율', '청소년비행_비율', '1인가구_비율',
       '1인가구_비율(여성)', '1인가구_비율(남성)', '외국인비율', '총범죄율', '강력범죄율', '가구-계 (가구)',
       '정신의료기관수', '정신의료기관 병상 수', '정신의료기관비율', '병상 수 비율', '우울감경험율', '2023 실업률'],
      dtype='object')